In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('CBS_2021-2023_Full.csv', low_memory=False)

### Context

Capital bike share project. First step: investigating aggregated DAILY weather and HOURLY weather data. 

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member


### Features - for FULL data

- **ride_id**	    = OBJECT [string of numbers and letters] 
- **rideable_type**	= OBJECT [classic bike, electric bike]
- **started_at**	= OBJECT->Changed to DATE [DATE + TIME]	
- **ended_at**	= OBJECT->Changed to DATE [DATE + TIME]	
- **start_station_name**	= OBJECT [Address]	
- **start_station_id**	= OBJECT [5-digit-number with one decimal]	
- **end_station_name**	= OBJECT [Address]	
- **end_station_id**	= OBJECT [5-digit-number with one decimal]	
- **start_lat**	= Float [38.76 to 39.14]	
- **start_lng**	= Float [-77.40 to -76.82]	
- **end_lat**	= Float [0.00 to 39.19]	
- **end_lng**	= Float [-77.56 to 0.00]	
- **member_casual**	= OBJECT [casual or member]

- **date**	= datetime64[ns]
- **year**	= int32
- **season**	= object
- **month**	= object
- **day_of_week**	= object
- **weekday**	= object
- **public_holiday**	= object
- **start_end_station**	= object
- **duration_min**	= timedelta64[ns]
- **start_end_station_id**	= object

In [4]:
# Check info and dtypes of the dataframe
print("Info of the DataFrame:\n", '\n')
df.info()

# observation: "Casual" & "Total_rides") are int64, but it should be float64?
# date: Object -> change to datetime

Info of the DataFrame:
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693997 entries, 0 to 10693996
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 1.0+ GB


In [5]:
# Check unique values before conversion
unique_values_before = df.nunique()
print("Unique values before conversion:\n", unique_values_before, '\n')

# 11 rows with missing data? Variance between 26.280 and 26.269 non-nulls

Unique values before conversion:
 ride_id               10693997
rideable_type                3
started_at             9649173
ended_at               9638365
start_station_name         860
start_station_id          1321
end_station_name           865
end_station_id            1338
start_lat               570110
start_lng               617671
end_lat                 173835
end_lng                 183886
member_casual                2
dtype: int64 



In [6]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
start_lat,10693995.0,38.903306,0.026842,38.76,38.890539,38.902760,38.914751,39.14
start_lng,10693995.0,-77.031872,0.033462,-77.40,-77.044609,-77.031617,-77.013667,-76.82
end_lat,10677232.0,38.902256,0.050493,0.00,38.890496,38.902314,38.912648,39.19
end_lng,10677232.0,-77.031463,0.091278,-77.56,-77.044661,-77.031500,-77.012808,0.00


In [7]:
# Check Memory usage before converting
memory_usage = df.memory_usage(deep=True)
print("Memory Usage before Converting dtypes:\n", memory_usage, '\n')

Memory Usage before Converting dtypes:
 Index                       128
ride_id               780661781
rideable_type         739790470
started_at            812743772
ended_at              812743772
start_station_name    820106397
start_station_id      660578762
end_station_name      815981136
end_station_id        658388628
start_lat              85551976
start_lng              85551976
end_lat                85551976
end_lng                85551976
member_casual         673721811
dtype: int64 



In [8]:
# Copy the dataframe
df_full = df.copy()

# Convert categorical columns to category type for memory efficiency
df_full["rideable_type"] = df_full["rideable_type"].astype("category")
df_full["start_station_name"] = df_full["start_station_name"].astype("category")
df_full["end_station_name"] = df_full["end_station_name"].astype("category")
df_full["member_casual"] = df_full["member_casual"].astype("category")

# Convert started_at and ended_at to datetime for memory efficiency
df_full['started_at'] = pd.to_datetime(df_full['started_at'])
df_full['ended_at'] = pd.to_datetime(df_full['ended_at'])

# Regenerate the entire ride_id column with sequential integers and convert it
# to a memory-efficient data type (uint32)
df_full['ride_id'] = pd.Series(range(1, len(df_full) + 1), dtype='uint32')

In [9]:
# Check info and dtypes after conversion
info_after = df_full.info()
print("Info of the DataFrame after conversion:\n", info_after, '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693997 entries, 0 to 10693996
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             uint32        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  category      
 5   start_station_id    object        
 6   end_station_name    category      
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       category      
dtypes: category(4), datetime64[ns](2), float64(4), object(2), uint32(1)
memory usage: 754.8+ MB
Info of the DataFrame after conversion:
 None 



In [10]:
# Check unique values after conversion
unique_values_after = df_full.nunique()
print("Unique values after conversion:\n", unique_values_after, '\n')

Unique values after conversion:
 ride_id               10693997
rideable_type                3
started_at             9649173
ended_at               9638365
start_station_name         860
start_station_id          1321
end_station_name           865
end_station_id            1338
start_lat               570110
start_lng               617671
end_lat                 173835
end_lng                 183886
member_casual                2
dtype: int64 



In [11]:
# Check memory usage after all conversions
print("Memory Usage after Converting dtype:\n", df_full.memory_usage(deep=True), '\n')

Memory Usage after Converting dtype:
 Index                       128
ride_id                42775988
rideable_type          10694312
started_at             85551976
ended_at               85551976
start_station_name     21492089
start_station_id      660578762
end_station_name       21492487
end_station_id        658388628
start_lat              85551976
start_lng              85551976
end_lat                85551976
end_lng                85551976
member_casual          10694231
dtype: int64 



In [12]:
# Calculate how much we reduced the in-memory footprint of the dataset
reduction = df_full.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()
print(f"We reduced the in-memory footprint of the dataset to {reduction:.2f} of its original size.")


We reduced the in-memory footprint of the dataset to 0.27 of its original size.


Creation of new columns

In [13]:
# new columns date to be created using started_at
df_full['date'] = df_full['started_at'].dt.date
df_full['date'] = pd.to_datetime(df_full['date'])
df_full.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,1,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member,2021-01-01
1,2,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual,2021-01-01
2,3,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member,2021-01-01
3,4,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member,2021-01-01
4,5,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member,2021-01-01


In [14]:
import pandas as pd
import datetime as dt

# Create a new column with the year
df_full['year'] = df_full['date'].dt.year

# Create a new column with the season
df_full['season'] = df_full['date'].dt.quarter
# Transform the season into a string
df_full['season'] = df_full['season'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'})

# Create a new column with the month
df_full['month'] = df_full['date'].dt.month
# Transform the month into a string
df_full['month'] = df_full['month'].map({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})

# Create a new column with the year
df_full['year'] = df_full['date'].dt.year

# Create a new column with the week number
df_full['week_number'] = df_full['date'].dt.isocalendar().week

# Create a new column with the day of the week
df_full['day_of_week'] = df_full['date'].dt.day_name()

# Create a new column with weekday yes no
df_full['weekday'] = np.where(df_full['day_of_week'].isin(['Saturday', 'Sunday']), 'no', 'yes')

# Create a new column with public holiday yes no USA
df_full['public_holiday'] = np.where(df_full['date'].isin(['2021-01-01', '2021-01-18', '2021-02-15', '2021-05-31', '2021-07-04', '2021-09-06', '2021-10-11', '2021-11-11', '2021-11-25', '2021-12-25',
                                                                         '2022-01-01', '2022-01-17', '2022-02-21', '2022-05-30', '2022-07-04', '2022-09-05', '2022-10-10', '2022-11-11', '2022-11-24', '2022-12-25',
                                                                         '2023-01-01', '2023-01-16', '2023-02-20', '2023-05-29', '2023-07-04', '2023-09-04', '2023-10-09', '2023-11-11', '2023-11-23', '2023-12-25']), 'yes', 'no')

df_full.head()

C:\Users\MP_Laptop\AppData\Local\Temp\ipykernel_11804\1293139667.py:30: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_full['public_holiday'] = np.where(df_full['date'].isin(['2021-01-01', '2021-01-18', '2021-02-15', '2021-05-31', '2021-07-04', '2021-09-06', '2021-10-11', '2021-11-11', '2021-11-25', '2021-12-25',


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,date,year,season,month,week_number,day_of_week,weekday,public_holiday
0,1,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,...,-77.072305,member,2021-01-01,2021,Winter,January,53,Friday,yes,yes
1,2,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,...,-77.020000,casual,2021-01-01,2021,Winter,January,53,Friday,yes,yes
2,3,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,...,-77.031691,member,2021-01-01,2021,Winter,January,53,Friday,yes,yes
3,4,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,...,-77.114563,member,2021-01-01,2021,Winter,January,53,Friday,yes,yes
4,5,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,...,-77.032956,member,2021-01-01,2021,Winter,January,53,Friday,yes,yes


In [15]:
# Create new column which combines start station and end station
df_full['start_end_station'] = df_full['start_station_name'].astype(str) + ' - ' + df_full['end_station_name'].astype(str)

# Create new column with start station id and end station id
df_full['start_end_station_id'] = df_full['start_station_id'].astype(str) + ' - ' + df_full['end_station_id'].astype(str)

# Create new column with the duration of the ride in minutes
df_full['duration_min'] = (df_full['ended_at'] - df_full['started_at']).dt.total_seconds() / 60

# df['period'] = df[['Year', 'quarter', ...]].agg('-'.join, axis=1)

# Create a plot with start_end_station_id and number of rides
# limit graph to 20 stations with highest number of rides to make it more readable
# plt.figure(figsize=(20, 10))
# sns.countplot(data=df_full, x='start_end_station_id', order=df_full['start_end_station_id'].value_counts().index(max=20))
# plt.xticks(rotation=90)

In [16]:
df_full.sort_values(by='duration_min', ascending=False).head(20)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,year,season,month,week_number,day_of_week,weekday,public_holiday,start_end_station,start_end_station_id,duration_min
578848,578849,docked_bike,2021-05-02 12:25:01,2021-06-09 09:25:29,17th & K St NW,31213.0,Ballston Metro / Stuart St & 9th St N,31037.0,38.902760,-77.038630,...,2021,Spring,May,17,Sunday,no,no,17th & K St NW - Ballston Metro / Stuart St & ...,31213.0 - 31037.0,54540.466667
9072162,9072163,docked_bike,2023-08-29 11:06:04,2023-10-05 02:00:19,Ohio Dr & West Basin Dr SW / MLK & FDR Memorials,31240.0,NaN,NaN,38.884120,-77.046570,...,2023,Summer,August,35,Tuesday,yes,no,Ohio Dr & West Basin Dr SW / MLK & FDR Memoria...,31240.0 - nan,52734.250000
3984327,3984328,docked_bike,2022-06-06 19:07:50,2022-07-13 08:44:41,USDA / 12th & C St SW,31217.0,6035 Warehouse,32901.0,38.886277,-77.028242,...,2022,Spring,June,23,Monday,yes,no,USDA / 12th & C St SW - 6035 Warehouse,31217.0 - 32901.0,52656.850000
8719173,8719174,docked_bike,2023-08-05 16:16:29,2023-09-08 16:44:28,Anacostia Roller Skating Pavilion,31720.0,22nd & P ST NW,31285.0,38.878771,-76.970930,...,2023,Summer,August,31,Saturday,no,no,Anacostia Roller Skating Pavilion - 22nd & P S...,31720.0 - 31285.0,48987.983333
866928,866929,docked_bike,2021-06-06 11:31:30,2021-07-10 10:51:17,Anacostia Roller Skating Pavillion,31720.0,Anacostia Roller Skating Pavillion,31720.0,38.878771,-76.970930,...,2021,Spring,June,22,Sunday,no,no,Anacostia Roller Skating Pavillion - Anacostia...,31720.0 - 31720.0,48919.783333
8869656,8869657,docked_bike,2023-08-16 10:18:38,2023-09-19 04:37:48,North Shore Dr & Village Rd,32223.0,NaN,NaN,38.970412,-77.340787,...,2023,Summer,August,33,Wednesday,yes,no,North Shore Dr & Village Rd - nan,32223.0 - nan,48619.166667
8278918,8278919,docked_bike,2023-07-07 18:08:28,2023-08-10 04:58:06,Maine Ave & Water St SW,31674.0,NaN,NaN,38.876747,-77.020945,...,2023,Summer,July,27,Friday,yes,no,Maine Ave & Water St SW - nan,31674.0 - nan,48169.633333
871700,871701,docked_bike,2021-06-06 18:49:47,2021-07-09 08:19:52,23rd & E St NW,31260.0,New Hampshire Ave & 24th St NW,31275.0,38.896104,-77.049882,...,2021,Spring,June,22,Sunday,no,no,23rd & E St NW - New Hampshire Ave & 24th St NW,31260.0 - 31275.0,46890.083333
824374,824375,docked_bike,2021-06-01 17:11:28,2021-07-02 17:58:16,8th & O St NW,31281.0,Georgia Ave & Emerson St NW,31405.0,38.908640,-77.022770,...,2021,Spring,June,22,Tuesday,yes,no,8th & O St NW - Georgia Ave & Emerson St NW,31281.0 - 31405.0,44686.800000
610600,610601,docked_bike,2021-05-06 20:02:49,2021-06-05 17:36:54,15th St & Constitution Ave NW,31321.0,3rd & M St NE,31627.0,38.892244,-77.033234,...,2021,Spring,May,18,Thursday,yes,no,15th St & Constitution Ave NW - 3rd & M St NE,31321.0 - 31627.0,43054.083333


Creation of "trip_distance" attribute using haversine

In [17]:
import numpy as np

def haversine_np(start_lng, start_lat, end_lng, end_lat):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    
    All args must be of equal length.    
    
    """
    start_lng, start_lat, end_lng, end_lat = map(np.radians, [start_lng, start_lat, end_lng, end_lat])
    
    dlon = end_lng - start_lng
    dlat = end_lat - start_lat
    
    a = np.sin(dlat/2.0)**2 + np.cos(start_lat) * np.cos(end_lat) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km

df_full['trip_distance'] = haversine_np(df_full['start_lng'],df_full['start_lat'],df_full['end_lng'],df_full['end_lat'])

df_full.head(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,season,month,week_number,day_of_week,weekday,public_holiday,start_end_station,start_end_station_id,duration_min,trip_distance
0,1,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,...,Winter,January,53,Friday,yes,yes,Maine Ave & 9th St SW - Rosslyn Metro / Wilson...,31646.0 - 31015.0,25.333333,4.372485
1,2,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,...,Winter,January,53,Friday,yes,yes,10th & U St NW - nan,31111.0 - nan,15.850000,4.792542
2,3,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,...,Winter,January,53,Friday,yes,yes,17th & Corcoran St NW - 14th & Belmont St NW,31214.0 - 31119.0,14.216667,1.140118
3,4,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,...,Winter,January,53,Friday,yes,yes,Wilson Blvd. & N. Vermont St. - Wilson Blvd. &...,31926.0 - 31926.0,5.583333,0.000000
4,5,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,...,Winter,January,53,Friday,yes,yes,11th & Park Rd NW - 14th & Newton St NW,31651.0 - 31649.0,3.233333,0.414513
5,6,electric_bike,2021-01-01 00:18:16,2021-01-01 00:29:05,NaN,NaN,Georgia Ave & Kennedy St NW,31415.0,38.930000,-77.030000,...,Winter,January,53,Friday,yes,yes,nan - Georgia Ave & Kennedy St NW,nan - 31415.0,10.816667,2.981746
6,7,classic_bike,2021-01-01 00:19:06,2021-01-01 00:36:28,Bladensburg Rd & Benning Rd NE,31617.0,Good Hope Rd & MLK Ave SE,31802.0,38.900413,-76.982872,...,Winter,January,53,Friday,yes,yes,Bladensburg Rd & Benning Rd NE - Good Hope Rd ...,31617.0 - 31802.0,17.366667,3.705148
7,8,electric_bike,2021-01-01 00:20:40,2021-01-01 00:35:36,Columbia Rd & Georgia Ave NW,31115.0,NaN,NaN,38.928081,-77.023768,...,Winter,January,53,Friday,yes,yes,Columbia Rd & Georgia Ave NW - nan,31115.0 - nan,14.933333,3.142934
8,9,docked_bike,2021-01-01 00:21:19,2021-01-01 00:33:31,15th & F St NE,31632.0,1st & M St SE,31650.0,38.897195,-76.983575,...,Winter,January,53,Friday,yes,yes,15th & F St NE - 1st & M St SE,31632.0 - 31650.0,12.200000,2.984562
9,10,classic_bike,2021-01-01 00:21:39,2021-01-01 00:31:07,14th & Otis Pl NW,31131.0,14th & R St NW,31202.0,38.934405,-77.032687,...,Winter,January,53,Friday,yes,yes,14th & Otis Pl NW - 14th & R St NW,31131.0 - 31202.0,9.466667,2.378400


In [18]:
# create a new column in dataset same_day_ride which checks if start and end day are the same and returns boolean
df_full['same_day_ride'] = np.where(df_full['started_at'].dt.day == df_full['ended_at'].dt.day, 'Yes', 'No')
df_full.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,month,week_number,day_of_week,weekday,public_holiday,start_end_station,start_end_station_id,duration_min,trip_distance,same_day_ride
0,1,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,...,January,53,Friday,yes,yes,Maine Ave & 9th St SW - Rosslyn Metro / Wilson...,31646.0 - 31015.0,25.333333,4.372485,Yes
1,2,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,...,January,53,Friday,yes,yes,10th & U St NW - nan,31111.0 - nan,15.850000,4.792542,Yes
2,3,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,...,January,53,Friday,yes,yes,17th & Corcoran St NW - 14th & Belmont St NW,31214.0 - 31119.0,14.216667,1.140118,Yes
3,4,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,...,January,53,Friday,yes,yes,Wilson Blvd. & N. Vermont St. - Wilson Blvd. &...,31926.0 - 31926.0,5.583333,0.000000,Yes
4,5,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,...,January,53,Friday,yes,yes,11th & Park Rd NW - 14th & Newton St NW,31651.0 - 31649.0,3.233333,0.414513,Yes
5,6,electric_bike,2021-01-01 00:18:16,2021-01-01 00:29:05,NaN,NaN,Georgia Ave & Kennedy St NW,31415.0,38.930000,-77.030000,...,January,53,Friday,yes,yes,nan - Georgia Ave & Kennedy St NW,nan - 31415.0,10.816667,2.981746,Yes
6,7,classic_bike,2021-01-01 00:19:06,2021-01-01 00:36:28,Bladensburg Rd & Benning Rd NE,31617.0,Good Hope Rd & MLK Ave SE,31802.0,38.900413,-76.982872,...,January,53,Friday,yes,yes,Bladensburg Rd & Benning Rd NE - Good Hope Rd ...,31617.0 - 31802.0,17.366667,3.705148,Yes
7,8,electric_bike,2021-01-01 00:20:40,2021-01-01 00:35:36,Columbia Rd & Georgia Ave NW,31115.0,NaN,NaN,38.928081,-77.023768,...,January,53,Friday,yes,yes,Columbia Rd & Georgia Ave NW - nan,31115.0 - nan,14.933333,3.142934,Yes
8,9,docked_bike,2021-01-01 00:21:19,2021-01-01 00:33:31,15th & F St NE,31632.0,1st & M St SE,31650.0,38.897195,-76.983575,...,January,53,Friday,yes,yes,15th & F St NE - 1st & M St SE,31632.0 - 31650.0,12.200000,2.984562,Yes
9,10,classic_bike,2021-01-01 00:21:39,2021-01-01 00:31:07,14th & Otis Pl NW,31131.0,14th & R St NW,31202.0,38.934405,-77.032687,...,January,53,Friday,yes,yes,14th & Otis Pl NW - 14th & R St NW,31131.0 - 31202.0,9.466667,2.378400,Yes


In [19]:
df_full['same_day_ride'].value_counts()

same_day_ride
Yes    10623984
No        70013
Name: count, dtype: int64

In [20]:
# Delete all rows where same_day_ride is 'No'
df_full = df_full[df_full['same_day_ride'] == 'Yes']


In [21]:
df_full['year'].value_counts()

year
2023    4441360
2022    3452941
2021    2729683
Name: count, dtype: int64

In [22]:
# Show all columns 
df_full.info()
# Check for missing values in the dataset
df_full.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 10623984 entries, 0 to 10693989
Data columns (total 26 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ride_id               uint32        
 1   rideable_type         category      
 2   started_at            datetime64[ns]
 3   ended_at              datetime64[ns]
 4   start_station_name    category      
 5   start_station_id      object        
 6   end_station_name      category      
 7   end_station_id        object        
 8   start_lat             float64       
 9   start_lng             float64       
 10  end_lat               float64       
 11  end_lng               float64       
 12  member_casual         category      
 13  date                  datetime64[ns]
 14  year                  int32         
 15  season                object        
 16  month                 object        
 17  week_number           UInt32        
 18  day_of_week           object        
 19  wee

ride_id                      0
rideable_type                0
started_at                   0
ended_at                     0
start_station_name      738642
start_station_id        738642
end_station_name        791281
end_station_id          791281
start_lat                    2
start_lng                    2
end_lat                   3801
end_lng                   3801
member_casual                0
date                         0
year                         0
season                       0
month                        0
week_number                  0
day_of_week                  0
weekday                      0
public_holiday               0
start_end_station            0
start_end_station_id         0
duration_min                 0
trip_distance             3801
same_day_ride                0
dtype: int64

In [23]:
df_full.groupby(['year', 'week_number']).size().unstack(level=0)

year,2021,2022,2023
week_number,,,
1,23368.0,15000.0,49798.0
2,25860.0,26675.0,44100.0
3,23678.0,26834.0,44505.0
4,18426.0,27587.0,45531.0
5,19918.0,29508.0,41060.0
6,17265.0,41853.0,52896.0
7,15861.0,37781.0,53635.0
8,24119.0,42933.0,56877.0
9,27658.0,52956.0,56827.0


Aggregation by day

In [24]:
# exclude docked bikes if the column exists
if 'rideable_type' in df_full.columns:
    df_full = df_full[df_full['rideable_type'] != 'docked_bike']

# check count of start stations per day
df_2 = df_full.copy() # to make sure any changes in df_full will not automatically affect df_full_2

# Create a new dataframe with unique dates and stations
unique_dates = df_2['date'].unique()
unique_stations = df_2['end_station_name'].unique()

# Extract unique end_station_name with their corresponding latitudes and longitudes
unique_stations_lat_lng = df_2[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates().rename(columns={'end_station_name': 'STATION', 'end_lat': 'LAT', 'end_lng': 'LNG'})

# Create df_2_unique with only existing combinations of dates and stations
df_2_unique = df_2[['date', 'end_station_name']].drop_duplicates().rename(columns={'date': 'DATE', 'end_station_name': 'STATION'})

# Add the 'weekday' column directly behind each date
df_2_unique = df_2_unique.merge(df_2[['date', 'weekday']].drop_duplicates(), left_on='DATE', right_on='date', how='left', suffixes=('', '_weekday'))
df_2_unique.rename(columns={'weekday': 'WEEKDAY'}, inplace=True)


# Calculate COUNT_PICKUP and COUNT_RETURN
count_pickup = df_2.groupby(['date', 'start_station_name']).size().reset_index(name='COUNT_PICKUP')
count_return = df_2.groupby(['date', 'end_station_name']).size().reset_index(name='COUNT_RETURN')

# Merge the counts into df_2_unique
df_2_unique = df_2_unique.merge(count_pickup, left_on=['DATE', 'STATION'], right_on=['date', 'start_station_name'], how='left', suffixes=('', '_pickup'))
df_2_unique = df_2_unique.merge(count_return, left_on=['DATE', 'STATION'], right_on=['date', 'end_station_name'], how='left', suffixes=('', '_return'))

# Fill NaN values with 0
df_2_unique['COUNT_PICKUP'] = df_2_unique['COUNT_PICKUP'].fillna(0)
df_2_unique['COUNT_RETURN'] = df_2_unique['COUNT_RETURN'].fillna(0)

# Simplify the overview to four columns
df_2_unique = df_2_unique[['DATE', 'WEEKDAY', 'STATION', 'COUNT_PICKUP', 'COUNT_RETURN']]

# Add a new column called 'BALANCE' at the end which by line subtracts COUNT_RETURN from COUNT_PICKUP
df_2_unique['BALANCE'] = df_2_unique['COUNT_RETURN'] - df_2_unique['COUNT_PICKUP']

# Remove all rows with empty value in column 'STATION'
df_2_unique = df_2_unique.dropna(subset=['STATION'])

df_2_unique

C:\Users\MP_Laptop\AppData\Local\Temp\ipykernel_11804\83482483.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_pickup = df_2.groupby(['date', 'start_station_name']).size().reset_index(name='COUNT_PICKUP')
C:\Users\MP_Laptop\AppData\Local\Temp\ipykernel_11804\83482483.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_return = df_2.groupby(['date', 'end_station_name']).size().reset_index(name='COUNT_RETURN')


,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
0,2021-01-01,yes,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,1.0,4.0,3.0
2,2021-01-01,yes,14th & Belmont St NW,18.0,10.0,-8.0
3,2021-01-01,yes,Wilson Blvd. & N. Vermont St.,3.0,3.0,0.0
4,2021-01-01,yes,14th & Newton St NW,6.0,3.0,-3.0
5,2021-01-01,yes,Georgia Ave & Kennedy St NW,0.0,2.0,2.0
...,...,...,...,...,...,...
589606,2023-12-31,no,N Shore Dr & Wainwright Dr,0.0,1.0,1.0
589607,2023-12-31,no,31st St & S Woodrow St,0.0,3.0,3.0
589608,2023-12-31,no,S Kenmore St & 24th St S,0.0,1.0,1.0
589609,2023-12-31,no,28th St & S Meade St,5.0,1.0,-4.0


Addition of capacity

In [25]:
# import csv file capacity_data.csv
df_capacity = pd.read_csv('../capacity_data.csv', sep=';')
df_capacity.tail()

,start_station_name,capacity,LAT,LNG
832,Radford St & Osage St,15,38.828437,-77.086031
833,Montgomery & East Ln,15,38.983525,-77.095367
834,Marion Barry Ave & Naylor Rd SE,15,38.861663,-76.969184
835,Motivate BX Tech office,15,38.964406,-77.010759
836,tech trailer V-1,15,38.964459,-77.010759


In [26]:
# compare row values in column start_station_name of dataframe df_capacity with row values in column STATION of dataframe df_cumulative_balance and print the mis-matches
# Compare STATION in df_cumulative_balance with start_station_name in df_capacity and print mismatches
mismatches = df_2_unique[~df_2_unique['STATION'].isin(df_capacity['start_station_name'])]
# print("Mismatches:\n", mismatches)
mismatches

,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
0,2021-01-01,yes,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,1.0,4.0,3.0
3,2021-01-01,yes,Wilson Blvd. & N. Vermont St.,3.0,3.0,0.0
10,2021-01-01,yes,Connecticut Ave & Newark St NW / Cleveland Park,3.0,2.0,-1.0
26,2021-01-01,yes,4th St & K St NW,2.0,4.0,2.0
48,2021-01-01,yes,18th St & S Eads St,1.0,2.0,1.0
...,...,...,...,...,...,...
434016,2023-04-24,yes,14th & Rhode Island Ave NW,55.0,59.0,4.0
434511,2023-04-25,yes,14th & Rhode Island Ave NW,71.0,72.0,1.0
435039,2023-04-26,yes,14th & Rhode Island Ave NW,70.0,66.0,-4.0
435663,2023-04-27,yes,14th & Rhode Island Ave NW,77.0,96.0,19.0


In [27]:
# Filter out records in df_2_unique that match the mismatches' STATION and DATE
df_2_unique = df_2_unique[~df_2_unique.set_index(['DATE', 'STATION']).index.isin(mismatches.set_index(['DATE', 'STATION']).index)]

# Display the result
df_2_unique

,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
2,2021-01-01,yes,14th & Belmont St NW,18.0,10.0,-8.0
4,2021-01-01,yes,14th & Newton St NW,6.0,3.0,-3.0
5,2021-01-01,yes,Georgia Ave & Kennedy St NW,0.0,2.0,2.0
6,2021-01-01,yes,Good Hope Rd & MLK Ave SE,1.0,1.0,0.0
7,2021-01-01,yes,14th & R St NW,5.0,7.0,2.0
...,...,...,...,...,...,...
589606,2023-12-31,no,N Shore Dr & Wainwright Dr,0.0,1.0,1.0
589607,2023-12-31,no,31st St & S Woodrow St,0.0,3.0,3.0
589608,2023-12-31,no,S Kenmore St & 24th St S,0.0,1.0,1.0
589609,2023-12-31,no,28th St & S Meade St,5.0,1.0,-4.0


In [28]:
# Drop 'LAT' and 'LNG' columns from df_capacity before merging
df_capacity_filtered = df_capacity.drop(columns=['LAT', 'LNG'])

# Merge df_2_unique with df_capacity_filtered on the station name
df_2_unique = df_2_unique.merge(df_capacity_filtered, left_on='STATION', right_on='start_station_name', how='left')

# Rename the 'capacity' column to 'CAPACITY'
df_2_unique.rename(columns={'capacity': 'CAPACITY'}, inplace=True)

# Drop the redundant 'start_station_name', 'LAT', and 'LNG' columns
df_2_unique.drop(columns=['start_station_name', 'LAT', 'LNG'], inplace=True, errors='ignore')

df_2_unique

,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY
0,2021-01-01,yes,14th & Belmont St NW,18.0,10.0,-8.0,15
1,2021-01-01,yes,14th & Newton St NW,6.0,3.0,-3.0,19
2,2021-01-01,yes,Georgia Ave & Kennedy St NW,0.0,2.0,2.0,19
3,2021-01-01,yes,Good Hope Rd & MLK Ave SE,1.0,1.0,0.0,15
4,2021-01-01,yes,14th & R St NW,5.0,7.0,2.0,19
...,...,...,...,...,...,...,...
567989,2023-12-31,no,N Shore Dr & Wainwright Dr,0.0,1.0,1.0,12
567990,2023-12-31,no,31st St & S Woodrow St,0.0,3.0,3.0,11
567991,2023-12-31,no,S Kenmore St & 24th St S,0.0,1.0,1.0,12
567992,2023-12-31,no,28th St & S Meade St,5.0,1.0,-4.0,15


In [29]:
# calculate an 'IMBALANCE FACTOR' for each row in df_2_unique. Take value in 'BALANCE'and divide it by the value in column 'CAPACITY' in same. 
# Result of this calculation should be put as float with one decimal in the new column 'IMBALANCE FACTOR' 
df_2_unique['IMBALANCE_FACTOR'] = (df_2_unique['BALANCE'] / df_2_unique['CAPACITY']).round(1)
df_2_unique

,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY,IMBALANCE_FACTOR
0,2021-01-01,yes,14th & Belmont St NW,18.0,10.0,-8.0,15,-0.5
1,2021-01-01,yes,14th & Newton St NW,6.0,3.0,-3.0,19,-0.2
2,2021-01-01,yes,Georgia Ave & Kennedy St NW,0.0,2.0,2.0,19,0.1
3,2021-01-01,yes,Good Hope Rd & MLK Ave SE,1.0,1.0,0.0,15,0.0
4,2021-01-01,yes,14th & R St NW,5.0,7.0,2.0,19,0.1
...,...,...,...,...,...,...,...,...
567989,2023-12-31,no,N Shore Dr & Wainwright Dr,0.0,1.0,1.0,12,0.1
567990,2023-12-31,no,31st St & S Woodrow St,0.0,3.0,3.0,11,0.3
567991,2023-12-31,no,S Kenmore St & 24th St S,0.0,1.0,1.0,12,0.1
567992,2023-12-31,no,28th St & S Meade St,5.0,1.0,-4.0,15,-0.3


In [30]:
# calculation of turnover
df_2_unique['TURNOVER'] = (df_2_unique['COUNT_PICKUP'] / df_2_unique['CAPACITY']).round(1)
df_2_unique

,DATE,WEEKDAY,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY,IMBALANCE_FACTOR,TURNOVER
0,2021-01-01,yes,14th & Belmont St NW,18.0,10.0,-8.0,15,-0.5,1.2
1,2021-01-01,yes,14th & Newton St NW,6.0,3.0,-3.0,19,-0.2,0.3
2,2021-01-01,yes,Georgia Ave & Kennedy St NW,0.0,2.0,2.0,19,0.1,0.0
3,2021-01-01,yes,Good Hope Rd & MLK Ave SE,1.0,1.0,0.0,15,0.0,0.1
4,2021-01-01,yes,14th & R St NW,5.0,7.0,2.0,19,0.1,0.3
...,...,...,...,...,...,...,...,...,...
567989,2023-12-31,no,N Shore Dr & Wainwright Dr,0.0,1.0,1.0,12,0.1,0.0
567990,2023-12-31,no,31st St & S Woodrow St,0.0,3.0,3.0,11,0.3,0.0
567991,2023-12-31,no,S Kenmore St & 24th St S,0.0,1.0,1.0,12,0.1,0.0
567992,2023-12-31,no,28th St & S Meade St,5.0,1.0,-4.0,15,-0.3,0.3


In [31]:
# export df_2_unique to csv file
df_2_unique.to_csv('aggregation_by_day.csv', index=False)

In [32]:
# # Define the station groups and their corresponding file names
# station_groups = {
#     "TO_btw_3_and_5": ["10th & K St NW", "11th & M St NW"],
#     "TO_btw_1_and_1-5": ["10th & H St NE", "10th & U St NW"],
#     "TO_below_0-33": ["10th St & Rhode Island Ave NE", "12th & Varnum St NE"],
#     "IMB_FAC_btw_pos_3_and_5": ["Jefferson Dr & 14th St SW", "M St & New Jersey Ave SE"],
#     "IMB_FAC_btw_pos_0-5_and_neg_0-5": ["10th & H St NE", "10th & K St NW"],
#     "IMB_FAC_btw_neg_3_and_5": ["11th & O St NW", "11th & S St NW"]
# }

# # Iterate over each group and save the filtered data to CSV
# for group_name, stations in station_groups.items():
#     for station in stations:
#         # Filter the dataframe for the current station
#         filtered_df = df_2_unique[df_2_unique['STATION'] == station]
        
#         # Generate the file name by replacing spaces with underscores
#         file_name = f"{group_name}_{station.replace(' ', '_')}.csv"
        
#         # Save the filtered dataframe to a CSV file
#         filtered_df.to_csv(file_name, index=False)


Aggregation to weekly level

In [33]:
# Aggregate daily data in df_2_unique to weekly level using ISO calendar week and year

# exclude docked bikes if the column exists
if 'rideable_type' in df_full.columns:
    df_full = df_full[df_full['rideable_type'] != 'docked_bike']

# check count of start stations per day
df_3 = df_full.copy()  # to make sure any changes in df_full will not automatically affect df_3

# Create a new dataframe with unique dates and stations
unique_dates = df_3['date'].unique()
unique_stations = df_3['end_station_name'].unique()

# Extract unique end_station_name with their corresponding latitudes and longitudes
unique_stations_lat_lng = df_3[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates().rename(columns={'end_station_name': 'STATION', 'end_lat': 'LAT', 'end_lng': 'LNG'})

# Create df_3_unique with only existing combinations of dates and stations
df_3_unique = df_3[['date', 'end_station_name']].drop_duplicates().rename(columns={'date': 'DATE', 'end_station_name': 'STATION'})

# Add the 'weekday' column directly behind each date
df_3_unique = df_3_unique.merge(df_3[['date', 'weekday']].drop_duplicates(), left_on='DATE', right_on='date', how='left', suffixes=('', '_weekday'))
df_3_unique.rename(columns={'weekday': 'WEEKDAY'}, inplace=True)

# Calculate COUNT_PICKUP and COUNT_RETURN
count_pickup = df_3.groupby(['date', 'start_station_name']).size().reset_index(name='COUNT_PICKUP')
count_return = df_3.groupby(['date', 'end_station_name']).size().reset_index(name='COUNT_RETURN')

# Merge the counts into df_3_unique
df_3_unique = df_3_unique.merge(count_pickup, left_on=['DATE', 'STATION'], right_on=['date', 'start_station_name'], how='left', suffixes=('', '_pickup'))
df_3_unique = df_3_unique.merge(count_return, left_on=['DATE', 'STATION'], right_on=['date', 'end_station_name'], how='left', suffixes=('', '_return'))

# Fill NaN values with 0
df_3_unique['COUNT_PICKUP'] = df_3_unique['COUNT_PICKUP'].fillna(0)
df_3_unique['COUNT_RETURN'] = df_3_unique['COUNT_RETURN'].fillna(0)

# Simplify the overview to four columns
df_3_unique = df_3_unique[['DATE', 'WEEKDAY', 'STATION', 'COUNT_PICKUP', 'COUNT_RETURN']]

# Add a new column called 'BALANCE' at the end which by line subtracts COUNT_RETURN from COUNT_PICKUP
df_3_unique['BALANCE'] = df_3_unique['COUNT_RETURN'] - df_3_unique['COUNT_PICKUP']

# Remove all rows with empty value in column 'STATION'
df_3_unique = df_3_unique.dropna(subset=['STATION'])

# Add a new column for the year
df_3_unique['YEAR'] = df_3_unique['DATE'].dt.year

# Aggregate data to weekly level using ISO calendar week and year
df_3_unique['WEEK'] = df_3_unique['DATE'].dt.isocalendar().week
df_3_unique = df_3_unique.groupby(['YEAR', 'WEEK', 'STATION']).agg({
    'COUNT_PICKUP': 'sum',
    'COUNT_RETURN': 'sum',
    'BALANCE': 'sum'
}).reset_index()

# Reorder columns to make 'YEAR' and 'WEEK' the first columns
df_3_unique = df_3_unique[['YEAR', 'WEEK', 'STATION', 'COUNT_PICKUP', 'COUNT_RETURN', 'BALANCE']]

df_3_unique

C:\Users\MP_Laptop\AppData\Local\Temp\ipykernel_11804\1211542313.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_pickup = df_3.groupby(['date', 'start_station_name']).size().reset_index(name='COUNT_PICKUP')
C:\Users\MP_Laptop\AppData\Local\Temp\ipykernel_11804\1211542313.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_return = df_3.groupby(['date', 'end_station_name']).size().reset_index(name='COUNT_RETURN')


,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
0,2021,1,10th & E St NW,42.0,56.0,14.0
1,2021,1,10th & Florida Ave NW,104.0,110.0,6.0
2,2021,1,10th & G St NW,38.0,33.0,-5.0
3,2021,1,10th & K St NW,71.0,71.0,0.0
4,2021,1,10th & Monroe St NE,15.0,12.0,-3.0
...,...,...,...,...,...,...
102813,2023,52,Wisconsin Ave & Upton St NW,6.0,14.0,8.0
102814,2023,52,Woodglen Dr & Executive Blvd,8.0,10.0,2.0
102815,2023,52,Woodley Park Metro / Calvert St & Connecticut ...,130.0,123.0,-7.0
102816,2023,52,Woodmont Ave & Strathmore St,41.0,32.0,-9.0


Adding of capacity

In [34]:
# import csv file capacity_data.csv
df_capacity = pd.read_csv('../capacity_data.csv', sep=';')
df_capacity.tail()

,start_station_name,capacity,LAT,LNG
832,Radford St & Osage St,15,38.828437,-77.086031
833,Montgomery & East Ln,15,38.983525,-77.095367
834,Marion Barry Ave & Naylor Rd SE,15,38.861663,-76.969184
835,Motivate BX Tech office,15,38.964406,-77.010759
836,tech trailer V-1,15,38.964459,-77.010759


In [35]:
# compare row values in column start_station_name of dataframe df_capacity with row values in column STATION of dataframe df_cumulative_balance and print the mis-matches
# Compare STATION in df_cumulative_balance with start_station_name in df_capacity and print mismatches
mismatches = df_3_unique[~df_3_unique['STATION'].isin(df_capacity['start_station_name'])]
# print("Mismatches:\n", mismatches)
mismatches

,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
1,2021,1,10th & Florida Ave NW,104.0,110.0,6.0
11,2021,1,11th & H St NE,53.0,58.0,5.0
36,2021,1,14th & Luzon St NW,17.0,15.0,-2.0
41,2021,1,14th & Rhode Island Ave NW,125.0,120.0,-5.0
49,2021,1,15th & Euclid St NW,177.0,101.0,-76.0
...,...,...,...,...,...,...
74639,2023,14,14th & Rhode Island Ave NW,515.0,526.0,11.0
75339,2023,15,14th & Rhode Island Ave NW,533.0,536.0,3.0
76051,2023,16,14th & Rhode Island Ave NW,494.0,510.0,16.0
76757,2023,17,14th & Rhode Island Ave NW,286.0,301.0,15.0


In [36]:
# Filter out records in df_2_unique that match the mismatches' STATION and DATE
df_3_unique = df_3_unique[~df_3_unique.set_index(['YEAR', 'WEEK', 'STATION']).index.isin(mismatches.set_index(['YEAR', 'WEEK', 'STATION']).index)]

# Display the result
df_3_unique

,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE
0,2021,1,10th & E St NW,42.0,56.0,14.0
2,2021,1,10th & G St NW,38.0,33.0,-5.0
3,2021,1,10th & K St NW,71.0,71.0,0.0
4,2021,1,10th & Monroe St NE,15.0,12.0,-3.0
5,2021,1,10th & U St NW,70.0,63.0,-7.0
...,...,...,...,...,...,...
102813,2023,52,Wisconsin Ave & Upton St NW,6.0,14.0,8.0
102814,2023,52,Woodglen Dr & Executive Blvd,8.0,10.0,2.0
102815,2023,52,Woodley Park Metro / Calvert St & Connecticut ...,130.0,123.0,-7.0
102816,2023,52,Woodmont Ave & Strathmore St,41.0,32.0,-9.0


In [37]:
# Drop 'LAT' and 'LNG' columns from df_capacity before merging
df_capacity_filtered = df_capacity.drop(columns=['LAT', 'LNG'])

# Merge df_2_unique with df_capacity_filtered on the station name
df_3_unique = df_3_unique.merge(df_capacity_filtered, left_on='STATION', right_on='start_station_name', how='left')

# Rename the 'capacity' column to 'CAPACITY'
df_3_unique.rename(columns={'capacity': 'CAPACITY'}, inplace=True)

# Drop the redundant 'start_station_name', 'LAT', and 'LNG' columns
df_3_unique.drop(columns=['start_station_name', 'LAT', 'LNG'], inplace=True, errors='ignore')

df_3_unique

,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY
0,2021,1,10th & E St NW,42.0,56.0,14.0,15
1,2021,1,10th & G St NW,38.0,33.0,-5.0,23
2,2021,1,10th & K St NW,71.0,71.0,0.0,23
3,2021,1,10th & Monroe St NE,15.0,12.0,-3.0,19
4,2021,1,10th & U St NW,70.0,63.0,-7.0,19
...,...,...,...,...,...,...,...
99144,2023,52,Wisconsin Ave & Upton St NW,6.0,14.0,8.0,15
99145,2023,52,Woodglen Dr & Executive Blvd,8.0,10.0,2.0,16
99146,2023,52,Woodley Park Metro / Calvert St & Connecticut ...,130.0,123.0,-7.0,23
99147,2023,52,Woodmont Ave & Strathmore St,41.0,32.0,-9.0,11


Calculation of Imbalance Factor

In [38]:
# calculate an 'IMBALANCE FACTOR' for each row in df_2_unique. Take value in 'BALANCE'and divide it by the value in column 'CAPACITY' in same. 
# Result of this calculation should be put as float with one decimal in the new column 'IMBALANCE FACTOR' 
df_3_unique['IMBALANCE_FACTOR'] = (df_3_unique['BALANCE'] / df_3_unique['CAPACITY']).round(1)
df_3_unique

,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY,IMBALANCE_FACTOR
0,2021,1,10th & E St NW,42.0,56.0,14.0,15,0.9
1,2021,1,10th & G St NW,38.0,33.0,-5.0,23,-0.2
2,2021,1,10th & K St NW,71.0,71.0,0.0,23,0.0
3,2021,1,10th & Monroe St NE,15.0,12.0,-3.0,19,-0.2
4,2021,1,10th & U St NW,70.0,63.0,-7.0,19,-0.4
...,...,...,...,...,...,...,...,...
99144,2023,52,Wisconsin Ave & Upton St NW,6.0,14.0,8.0,15,0.5
99145,2023,52,Woodglen Dr & Executive Blvd,8.0,10.0,2.0,16,0.1
99146,2023,52,Woodley Park Metro / Calvert St & Connecticut ...,130.0,123.0,-7.0,23,-0.3
99147,2023,52,Woodmont Ave & Strathmore St,41.0,32.0,-9.0,11,-0.8


Calculation of Turnover

In [39]:
# calculation of turnover
df_3_unique['TURNOVER'] = ((df_3_unique['COUNT_PICKUP'] / 7) / df_3_unique['CAPACITY']).round(1)
df_3_unique

,YEAR,WEEK,STATION,COUNT_PICKUP,COUNT_RETURN,BALANCE,CAPACITY,IMBALANCE_FACTOR,TURNOVER
0,2021,1,10th & E St NW,42.0,56.0,14.0,15,0.9,0.4
1,2021,1,10th & G St NW,38.0,33.0,-5.0,23,-0.2,0.2
2,2021,1,10th & K St NW,71.0,71.0,0.0,23,0.0,0.4
3,2021,1,10th & Monroe St NE,15.0,12.0,-3.0,19,-0.2,0.1
4,2021,1,10th & U St NW,70.0,63.0,-7.0,19,-0.4,0.5
...,...,...,...,...,...,...,...,...,...
99144,2023,52,Wisconsin Ave & Upton St NW,6.0,14.0,8.0,15,0.5,0.1
99145,2023,52,Woodglen Dr & Executive Blvd,8.0,10.0,2.0,16,0.1,0.1
99146,2023,52,Woodley Park Metro / Calvert St & Connecticut ...,130.0,123.0,-7.0,23,-0.3,0.8
99147,2023,52,Woodmont Ave & Strathmore St,41.0,32.0,-9.0,11,-0.8,0.5


In [40]:
# Export df_3_unique to a CSV file
df_3_unique.to_csv('aggregation_by_week.csv', index=False)

In [ ]:
# Define the station groups and their corresponding file names
station_groups = {
    "W_TO_btw_3_and_5": ["10th & K St NW", "11th & M St NW"],
    "W_TO_btw_1_and_1-5": ["10th & H St NE", "10th & U St NW"],
    "W_TO_below_0-33": ["10th St & Rhode Island Ave NE", "12th & Varnum St NE"],
    "W_IMB_FAC_btw_pos_3_and_5": ["Jefferson Dr & 14th St SW", "M St & New Jersey Ave SE"],
    "W_IMB_FAC_btw_pos_0-5_and_neg_0-5": ["10th & H St NE", "10th & K St NW"],
    "W_IMB_FAC_btw_neg_3_and_5": ["11th & O St NW", "11th & S St NW"]
}

# Iterate over each group and save the filtered data to CSV
for group_name, stations in station_groups.items():
    for station in stations:
        # Filter the dataframe for the current station
        filtered_df = df_3_unique[df_3_unique['STATION'] == station]
        
        # Generate the file name by replacing spaces with underscores
        file_name = f"{group_name}_{station.replace(' ', '_')}.csv"
        
        # Save the filtered dataframe to a CSV file
        filtered_df.to_csv(file_name, index=False)
